# RAG 2: Fast and Free 
The second RAG pipeline leverages open source and local technologies, optimising for fast performance and free implementation.

In [1]:
from llama_index.core import Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.llms.ollama import Ollama
import weaviate
from utils import load_documents, get_nodes, create_index

DOCUMENTS_PATH = "./source_documents/"
INDEX_NAME = "InsurancePolicyIndex" 
COUNT_NODES_RETRIEVED = 2
DENSE_VECTOR_HYBRID_WEIGHTING = 0.5 

# Define Weaviate client 
client = weaviate.Client(embedded_options=weaviate.EmbeddedOptions())

# Delete existing index if exists
client.schema.delete_class(INDEX_NAME)

# Define and configure embedding and generation LLMs
Settings.embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5") # Set embedding model globally to index and retrieve using the same model 
generation_llm  = Ollama(model="gemma:2b", request_timeout=30.0)

# Create Retriever
documents = load_documents(DOCUMENTS_PATH)
nodes = get_nodes(documents)
vector_store = WeaviateVectorStore(weaviate_client=client, index_name=INDEX_NAME)
index = create_index(nodes, vector_store)
retriever = index.as_retriever()

# Configure Query Engine
query_engine = index.as_query_engine(
    vector_store_query_mode = "hybrid", 
    alpha = DENSE_VECTOR_HYBRID_WEIGHTING,
    llm = generation_llm,
    similarity_top_k = COUNT_NODES_RETRIEVED, 
)

Started /Users/mzhao/.cache/weaviate-embedded: process ID 35280


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-04-16T13:23:30+10:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-04-16T13:23:30+10:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-04-16T13:23:30+10:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-04-16T13:23:30+10:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-04-16T13:23:30+10:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://12

In [2]:
response = query_engine.query("What types of insurance are available?")
print(response)

Sure, the query is about the types of insurance available. Based on the context information, the following types of insurance are available:

• Comprehensive Plus Insurance
• Comprehensive Insurance
• Third Party Fire & Theft Insurance
• Third Party Property Damage Insurance


# Pipeline Evaluation

In [3]:
# Load OPENAI_API_KEY from .env file for evaluation
from dotenv import load_dotenv
load_dotenv()

True

### Retreival Evaluation

In [4]:
import nest_asyncio
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.llms.openai import OpenAI
from utils import display_retrieval_evaluation_results, create_retrieval_qa_dataset

# Allows for nested async calls in Jupyter notebooks
nest_asyncio.apply()

# Define evaluation LLM
evaluation_llm = OpenAI(temperature=0) # Ideally this should be a superior model to generation_llm (e.g. GPT-4), however due costs and rate limits, GPT-3.5 is used

# Create QA dataset
QA_DATASET_PATH = "./qa_datasets/qa_dataset_2.json"
qa_dataset = create_retrieval_qa_dataset(nodes, evaluation_llm, QA_DATASET_PATH)

# Evaluate QA dataset
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)
display_retrieval_evaluation_results(f"top-{COUNT_NODES_RETRIEVED} eval", eval_results)

Previous retrieval evaluation dataset deleted successfully.


  0%|          | 0/187 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 187/187 [03:03<00:00,  1.02it/s]


,retrievers,hit_rate,mrr
0,top-2 eval,0.691176,0.612745


### Generation Evaluation

#### Example evaluation

In [5]:
from llama_index.core.evaluation import FaithfulnessEvaluator, AnswerRelevancyEvaluator, ContextRelevancyEvaluator

# Example query and response 
query = "What types of insurance are available?"
response = query_engine.query(query)
print(response)

# Example Faithfulness evaluation
faithfulness_evaluator = FaithfulnessEvaluator(llm=evaluation_llm)
eval_result = faithfulness_evaluator.evaluate_response(query=query, response=response)
print("Faithfulness: " + str(eval_result.passing))

# Example Relevancy evaluation
relevancy_evaluator = AnswerRelevancyEvaluator(llm=evaluation_llm)
eval_result = relevancy_evaluator.evaluate_response(query=query, response=response)
print("Relevance: " + str(eval_result.score))

# Example Context evaluation
context_evaluator = ContextRelevancyEvaluator(llm=evaluation_llm)
eval_result = context_evaluator.evaluate_response(query=query, response=response)
print("Context: " + str(eval_result.score))

The types of insurance available are:

• Comprehensive Plus Insurance – our top cover
• Comprehensive Insurance – our standard cover
• Third Party Fire & Theft Insurance – our basic cover with some extras
• Third Party Property Damage Insurance – our basic cover
Faithfulness: True
Relevance: 1.0
Context: 1.0


#### Batch evaluation

In [6]:
from utils import (
    create_question_dataset, 
    create_prediction_dataset, 
    create_judges, 
    create_evaluation_tasks, 
    evaluate_tasks, 
    display_generation_evaluation_results
)

# Create rag question dataset    
rag_dataset = create_question_dataset(nodes, evaluation_llm)

In [7]:
# Create prediction dataset
prediction_data = await create_prediction_dataset(rag_dataset, query_engine)

Batch processing of predictions: 100%|██████████| 2/2 [00:12<00:00,  6.36s/it]


In [8]:
# Create Evaluation tasks using evaluation LLM judge
judges = create_judges(evaluation_llm)
eval_tasks = create_evaluation_tasks(rag_dataset, prediction_data, judges)

In [9]:
# Evaluate tasks to get evaluation results
eval_results = await evaluate_tasks(eval_tasks)

In [10]:
# Display evaluation results
display_generation_evaluation_results(eval_results)

rag,mean value
metrics,
mean_answer_relevancy_score,0.909091
mean_context_relevancy_score,0.869652
mean_faithfulness_score,0.834225
mean_semantic_similarity_score,0.904560
